# Usages of various PyTorch apis

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from IPython.display import display_markdown

def print_md(s):
    display_markdown(s, raw=True)

## Embeddings
[API Docs](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html)

In [2]:
# This example illustrates the usage of nn.Embedding
# Embedding is a lookup table -- to lookup a vector stored against a key (typically an integer index)
# You supply a bunch of keys, you get back corresponding bunch of vectors

# This example tries to lookup vectors corresponding to words in a sentence
# Setup: here are 100 english words. Index of 'how' is 0, index of 'to' is 1, ... index of 'two' is 99
v100 = ['how', 'to', 'her', 'at', 'up', 'see', 'in', 'thing', 'even', 'because', 'or', 
             'what', 'man', 'this', 'for', 'with', 'time', 'now', 'give', 'very', 'take', 'other', 
             'there', 'would', 'first', 'about', 'people', 'think', 'find', 'so', 'say', 'as', 
             'many', 'will', 'just', 'he', 'I', 'well', 'our', 'tell', 'out', 'have', 'can', 'its', 
             'make', 'get', 'if', 'than', 'use', 'that', 'new', 'also', 'from', 'by', 'his', 'year', 
             'do', 'some', 'the', 'no', 'a', 'those', 'she', 'come', 'one', 'their', 'more', 
             'these', 'all', 'go', 'and', 'could', 'him', 'into', 'only', 'who', 'of', 'it', 'your', 
             'not', 'you', 'here', 'when', 'on', 'which', 'then', 'know', 'them', 'my', 'me', 'we', 
             'want', 'they', 'like', 'look', 'day', 'way', 'but', 'be', 'two']

# Naturally this vocabulary is insufficient even for most common sentences. Map all out of vocabulary words to <unk>
# add <unk> to vocabulary at index 100. <unk> is the replacement for out of vocabulary words
v100.append('<unk>')

# create a dictionary to lookup word's index
w2i = {word: i for i, word in enumerate(v100)}

# define few sentences of fixed size. we wish to lookup embeddings for words in these sentences
# here are 5 sentences
sents_lang = [
    'this is my book',
    'those are your books',
    'what is your name',
    'i will be back',
    'go out and about'
]

# tranform to indexed representation of sentences
sents_indices = [[w2i.get(word, w2i['<unk>']) for word in sent.split()] for sent in sents_lang]

# print representations
print_md('#### Sentences represented as array of indices')
for s1, s2 in zip(sents_lang, sents_indices):
    print_md('*{}* => {}'.format(s1, s2))

# convert sents_word_indices to a tensor
sents_tensor = torch.tensor(sents_indices)
print_md('#### Sentences represented as a tensor')
print(sents_tensor)
print('Shape = {}'.format(sents_tensor.shape))

#### Sentences represented as array of indices

*this is my book* => [13, 100, 88, 100]

*those are your books* => [61, 100, 78, 100]

*what is your name* => [11, 100, 78, 100]

*i will be back* => [100, 33, 98, 100]

*go out and about* => [69, 40, 70, 25]

#### Sentences represented as a tensor

tensor([[ 13, 100,  88, 100],
        [ 61, 100,  78, 100],
        [ 11, 100,  78, 100],
        [100,  33,  98, 100],
        [ 69,  40,  70,  25]])
Shape = torch.Size([5, 4])


In [3]:
# create an embedding to lookup 10-dimensional vectors for each word
embedding_1 = nn.Embedding(num_embeddings=len(v100), embedding_dim=10)

In [4]:
# underneath embeddings are weights, initialized with random values ~ N(0, 1)
print_md("#### Embedding weights")
print("Shape of weights = {}".format(embedding_1.weight.shape))

# what's the embedding for the word 'who'
embed = embedding_1(torch.tensor([ w2i['who']]))
print_md("#### Embedding of 'who'")
print("{}\nShape = {}".format(embed, embed.shape))

# what's the embedding of 2nd sentence
embed = embedding_1(sents_tensor[1])
print_md("#### Embedding of the sentence: *{}*".format(sents_lang[1]))
print("{}\nShape = {}".format(embed, embed.shape))

# embeddings for all sentences
embed = embedding_1(sents_tensor)
print_md("#### Embeddings of all 5 sentences")
print("{}\nShape = {}".format(embed, embed.shape))

#### Embedding weights

Shape of weights = torch.Size([101, 10])


#### Embedding of 'who'

tensor([[ 0.6254, -0.9818,  0.2887,  1.6523, -0.2394,  0.1089,  0.4927,  0.7786,
         -0.2254, -1.3948]], grad_fn=<EmbeddingBackward>)
Shape = torch.Size([1, 10])


#### Embedding of the sentence: *those are your books*

tensor([[-1.1023,  0.3330,  1.2189,  0.1245,  1.2960, -1.4813, -1.6429, -0.5803,
         -0.3800,  0.8051],
        [-2.4322,  1.1259, -1.2660,  0.9573, -0.0586,  0.0960, -0.1468,  1.6027,
          0.1917, -0.6241],
        [ 0.5807,  1.2417,  1.2596,  1.3719,  2.0855, -0.0294, -0.4046, -2.0112,
          0.1942,  0.5176],
        [-2.4322,  1.1259, -1.2660,  0.9573, -0.0586,  0.0960, -0.1468,  1.6027,
          0.1917, -0.6241]], grad_fn=<EmbeddingBackward>)
Shape = torch.Size([4, 10])


#### Embeddings of all 5 sentences

tensor([[[ 0.9835,  0.9457, -0.4992,  0.8422, -0.3864, -1.8317, -1.9502,
           0.0258,  0.4130, -0.5283],
         [-2.4322,  1.1259, -1.2660,  0.9573, -0.0586,  0.0960, -0.1468,
           1.6027,  0.1917, -0.6241],
         [-0.6409, -0.0850, -0.3380, -1.0989,  0.3010, -0.1985,  0.6614,
          -0.1264, -0.6709,  3.0239],
         [-2.4322,  1.1259, -1.2660,  0.9573, -0.0586,  0.0960, -0.1468,
           1.6027,  0.1917, -0.6241]],

        [[-1.1023,  0.3330,  1.2189,  0.1245,  1.2960, -1.4813, -1.6429,
          -0.5803, -0.3800,  0.8051],
         [-2.4322,  1.1259, -1.2660,  0.9573, -0.0586,  0.0960, -0.1468,
           1.6027,  0.1917, -0.6241],
         [ 0.5807,  1.2417,  1.2596,  1.3719,  2.0855, -0.0294, -0.4046,
          -2.0112,  0.1942,  0.5176],
         [-2.4322,  1.1259, -1.2660,  0.9573, -0.0586,  0.0960, -0.1468,
           1.6027,  0.1917, -0.6241]],

        [[-0.7752, -0.0060, -1.1712, -1.5884,  1.0062, -0.7441,  0.5334,
          -0.1678,  0.5138,  0.7598

In [5]:
# A little sophisticated embedding layer
# Suppose the embeddings we want to lookup are to be linear transformed. This can be done by implementing a custom module wrapping nn.Embedding
# This module applies a linear transformation on word embeddings of size 'in_embedding_dim', the output is embeddings of size 'out_embedding_dim'
class CustomEmbedding1(nn.Module):
    def __init__(self, num_embeddings, in_embedding_dim, out_embedding_dim):
        super(CustomEmbedding1, self).__init__()
        self.embedding = nn.Embedding(num_embeddings, in_embedding_dim)
        self.linear = nn.Linear(in_features=in_embedding_dim, out_features=out_embedding_dim)
    
    def forward(self, input):
        return F.relu_(self.linear(self.embedding(input)))

cust_embedding_1 = CustomEmbedding1(num_embeddings=len(v100), in_embedding_dim=10, out_embedding_dim=5)

In [6]:
# what's the embedding for the word 'who'
cust_embed = cust_embedding_1(torch.tensor([ w2i['who']]))
print_md("#### Embedding of 'who'")
print("{}\nShape = {}".format(cust_embed, cust_embed.shape))

# what's the embedding of 2nd sentence
cust_embed = cust_embedding_1(sents_tensor[1])
print_md("#### Embedding of the sentence: *{}*".format(sents_lang[1]))
print("{}\nShape = {}".format(cust_embed, cust_embed.shape))

# embeddings for all sentences
cust_embed = cust_embedding_1(sents_tensor)
print_md("#### Embeddings of all 5 sentences")
print("{}\nShape = {}".format(cust_embed, cust_embed.shape))

#### Embedding of 'who'

tensor([[0.0995, 0.8117, 0.0000, 0.0000, 0.0000]], grad_fn=<ReluBackward1>)
Shape = torch.Size([1, 5])


#### Embedding of the sentence: *those are your books*

tensor([[0.3331, 0.2566, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.4187, 0.6632, 0.1346, 0.0000],
        [0.0000, 0.2336, 0.0000, 0.0000, 0.9989],
        [0.0000, 0.4187, 0.6632, 0.1346, 0.0000]], grad_fn=<ReluBackward1>)
Shape = torch.Size([4, 5])


#### Embeddings of all 5 sentences

tensor([[[1.1082, 0.0000, 0.3293, 0.0000, 0.6195],
         [0.0000, 0.4187, 0.6632, 0.1346, 0.0000],
         [0.6458, 1.0109, 0.0591, 0.0000, 0.3925],
         [0.0000, 0.4187, 0.6632, 0.1346, 0.0000]],

        [[0.3331, 0.2566, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.4187, 0.6632, 0.1346, 0.0000],
         [0.0000, 0.2336, 0.0000, 0.0000, 0.9989],
         [0.0000, 0.4187, 0.6632, 0.1346, 0.0000]],

        [[0.0000, 0.4211, 0.0000, 0.0780, 0.5354],
         [0.0000, 0.4187, 0.6632, 0.1346, 0.0000],
         [0.0000, 0.2336, 0.0000, 0.0000, 0.9989],
         [0.0000, 0.4187, 0.6632, 0.1346, 0.0000]],

        [[0.0000, 0.4187, 0.6632, 0.1346, 0.0000],
         [0.0000, 0.9825, 0.2333, 0.0000, 0.2550],
         [0.0000, 0.1848, 0.3427, 0.6653, 0.3540],
         [0.0000, 0.4187, 0.6632, 0.1346, 0.0000]],

        [[0.4105, 0.0000, 0.1131, 0.0000, 0.5536],
         [0.0000, 1.6048, 0.0000, 0.0000, 0.7310],
         [0.2285, 0.7793, 0.0000, 0.0000, 0.2601],
         [0.0000, 0.000